# 라이브러리

In [ ]:
! pip3 install koreanize_matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 22.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

# 데이터 불러오기

In [ ]:
log_2023_path = "/content/drive/MyDrive/코드잇_데이터분석_6기/프로젝트/데이터 분석 중급 프로젝트_1/원본 데이터/주제 1. 국내 채용시장 및 채용 플랫폼 이용패턴 분석/log_2023.csv"
log_2023_df = pd.read_csv(log_2023_path)
log_2023_df.head(2)

,Unnamed: 0,user_uuid,URL,timestamp,date,response_code,method
0,0,5ce8f5ca-3476-4623-a60c-00c98eef3b62,@user_id,2023-12-29 13:19:50.230356 UTC,2023-12-29,200,GET
1,1,5ce8f5ca-3476-4623-a60c-00c98eef3b62,api/users/notifications/mark_read?id=6425064&_...,2023-12-29 13:20:17.848762 UTC,2023-12-29,200,GET


In [ ]:
# 1. UTC 제거
log_2023_df["timestamp_clean"] = log_2023_df["timestamp"].str.replace(" UTC", "", regex=False)

# 2. datetime으로 변환
log_2023_df["timestamp_clean"] = pd.to_datetime(
    log_2023_df["timestamp_clean"],
    format='%Y-%m-%d %H:%M:%S.%f',
    errors='coerce'
)

# 3. NaT가 생긴 경우, 포맷이 다른 문자열을 다시 처리 (마이크로초 없는 경우 등)
mask_failed = log_2023_df["timestamp_clean"].isna()
log_2023_df.loc[mask_failed, "timestamp_clean"] = pd.to_datetime(
    log_2023_df.loc[mask_failed, "timestamp"].str.replace(" UTC", "", regex=False),
    format="%Y-%m-%d %H:%M:%S",
    errors="coerce"
)

# 4. 정리
log_2023_df["timestamp"] = log_2023_df["timestamp_clean"]
log_2023_df.drop(columns="timestamp_clean", inplace=True)

# 전처리 데이터 추출

In [ ]:
# UTM 파라미터 값을 가진 행만 추출
utm_2023_df = log_2023_df[log_2023_df['URL'].str.contains('utm_', na=False)]

# 'api/'로 시작하는 행만 추출
api_2023_df = log_2023_df[log_2023_df['URL'].str.startswith('api/', na=False)]

In [ ]:
utm_2023_df.to_csv('utm_2023_df.csv', index=False, encoding='utf-8-sig')
api_2023_df.to_csv('api_2023_df.csv', index=False, encoding='utf-8-sig')

In [ ]:
from urllib.parse import urlparse, parse_qs
from collections import Counter

#  URL 의미 기반 태깅 함수
def map_funnel_stage(url):
    url = str(url).lower()

    if pd.isna(url) or url.strip() == '':
        return '기타'

    if 'search' in url or 'keywords' in url or 'suggest' in url or 'q=' in url:
        return '검색'
    elif 'job_title' in url or 'jobs/id' in url:
        return '공고 보기'
    elif 'companies' in url or 'company_id' in url:
        return '기업 보기'
    elif 'template' in url or 'career' in url or 'experience' in url or 'profile' in url:
        return '이력서 작성'
    elif 'apply/step' in url:
        return '지원 시작'
    elif 'applications' in url:
        return '지원 완료'
    elif 'notifications/mark_read' in url:
        return '알림 확인'
    elif '@user_id' in url or 'follower' in url or 'following' in url:
        return '유저 프로필'
    elif 'setting' in url or 'timeline' in url:
        return '설정'
    elif 'signup' in url or 'password_reset' in url:
        return '회원가입'
    else:
        return '기타'

# URL 쿼리 파라미터 추출
def extract_query_keywords(url):
    try:
        parsed = urlparse(str(url))
        query = parse_qs(parsed.query)
        keywords = []
        for k, v in query.items():
            if k in ['q', 'keywords', 'name', 'type', 'job', 'offset']:
                keywords.append(f"{k}={','.join(v)}")
        return '&'.join(keywords)
    except:
        return ''

# 의미 기반 컬럼 생성
log_2023_df['funnel_stage'] = log_2023_df['URL'].apply(map_funnel_stage)
log_2023_df['query_tags'] = log_2023_df['URL'].apply(extract_query_keywords)

# 분석용 테이블 구성
behavior_table = log_2023_df[['user_uuid', 'timestamp', 'URL', 'funnel_stage', 'query_tags']].sort_values(by=['user_uuid', 'timestamp'])
behavior_table = behavior_table[
    ~behavior_table['URL'].str.contains('utm', case=False, na=False) &
    ~behavior_table['URL'].str.startswith('api/', na=False)
].reset_index(drop=True)

In [ ]:
behavior_table.to_csv('log_2023_df.csv', index=False, encoding='utf-8-sig')